In [3]:
import numpy as np
import pandas as pd


In [4]:
# import dataset

df = pd.read_csv("product_sales.csv")
# Drop random columns
df = df.drop(columns=["Unnamed: 107", "94"])
# Melt data and remove strings for week names
df = df.melt(id_vars=["Scode", "Pcode", "Price"], var_name="Week", value_name="Quantity Sold")
df["Week"] = df["Week"].str.extract('(\d+)').astype(int)

In [ ]:
def evolution_strategy_with_adaptive_sigma(mu, lambda_, generations, bounds, sigma, params_dim):
    """
    (mu, lambda)-ES with adaptive sigma using the 1/5 success rule.
    """

    # Parameters for sigma update
    c_inc = 1.2  # Increase factor
    c_dec = 0.85  # Decrease factor

    # Initialize a random population of lambda_ individuals
    population = np.random.uniform(
        low=[bound[0] for bound in bounds],
        high=[bound[1] for bound in bounds],
        size=(params_dim, lambda_)
    )

    for generation in range(generations):
        # Evaluate fitness for the population
        fitness = np.array([fitness_function(ind) for ind in population])
        
        # Select the top mu individuals as parents
        parents_indices = np.argsort(fitness)[:mu]  # Minimize fitness
        parents = population[parents_indices]
        
        # Generate lambda_ offspring through mutation
        offspring = []
        successes = 0  # Count successful offspring
        for _ in range(lambda_):
            parent = parents[np.random.randint(mu)]
            child = parent + np.random.normal(0, sigma, params_dim)
            
            # Ensure offspring stay within bounds
            child = np.clip(child, [bound[0] for bound in bounds], [bound[1] for bound in bounds])
            
            # Check if the offspring is better than the parent
            if fitness_function(child) < fitness_function(parent):
                successes += 1
            offspring.append(child)
        
        population = np.array(offspring)

        # Update sigma based on success rate
        success_rate = successes / lambda_
        if success_rate > 1/5:
            sigma *= c_inc
        else:
            sigma *= c_dec

    # Return the best individual and its fitness from the final population
    best_fitness = np.min([fitness_function(ind) for ind in population])
    best_solution = population[np.argmin([fitness_function(ind) for ind in population])]
    
    return best_solution, best_fitness


# Define your fitness function here (example is a placeholder)
def fitness_function(individual):
    # Replace this with your actual fitness function
    x, y = individual
    return x**2 + y**2  # Example: minimize the distance to (0, 0)

# Example parameters
mu = 5
lambda_ = 20
generations = 100
bounds = [(1, 10), (-10, 10)]  # Example bounds for two parameters
sigma = 0.5  # Mutation step size
params_dim = len(unique(df["Scode"]))

# Run the evolution strategy
best_solution, best_fitness = evolution_strategy(mu, lambda_, generations, bounds, sigma, params_dim, )
print(f"Best Solution: {best_solution}")
print(f"Best Fitness: {best_fitness}")


In [ ]:
import torch

# Example parameters
mu = 5
lambda_ = 20
generations = 100
interval = 5
freq_bound = (1, interval)
amt_bound = (0, 1000)
sigma_freq = 0.5  # Mutation step size for frequency
sigma_amt = 20.0  # Mutation step size for amount

# Define parameter dimensions
params_dim = len(df["Scode"].unique())

# Combined tensor for freqs and amts
bounds = torch.tensor([
    [freq_bound[0], freq_bound[1]],  # Bounds for frequency
    [amt_bound[0], amt_bound[1]]    # Bounds for amount
])

# Initialize the population: (lambda_, params_dim, 2)
# Slice [:, :, 0] will represent frequencies, [:, :, 1] will represent amounts
population = torch.empty((lambda_, params_dim, 2))
population[:, :, 0] = torch.rand((lambda_, params_dim)) * (freq_bound[1] - freq_bound[0]) + freq_bound[0]
population[:, :, 1] = torch.rand((lambda_, params_dim)) * (amt_bound[1] - amt_bound[0]) + amt_bound[0]

# Evolution process
for generation in range(generations):
    # Evaluate fitness for all individuals (vectorized operation)
    fitness = torch.tensor([fitness_function(population[i]) for i in range(lambda_)])

    # Select the top mu individuals based on fitness (minimization assumed)
    _, parents_indices = torch.topk(fitness, mu)  # Use positive fitness for maximization
    parents = population[parents_indices]  # Shape: (mu, params_dim, 2)

    # Generate lambda_ offspring via mutation
    offspring = parents[torch.randint(0, mu, (lambda_,))]  # Random parent selection
    noise = torch.empty_like(offspring)
    noise[:, :, 0] = torch.randn((lambda_, params_dim)) * sigma_freq  # Mutation for frequency
    noise[:, :, 1] = torch.randn((lambda_, params_dim)) * sigma_amt   # Mutation for amount
    offspring += noise

    # Clip offspring to ensure bounds
    offspring[:, :, 0] = torch.clip(offspring[:, :, 0], freq_bound[0], freq_bound[1])  # Clip frequency
    offspring[:, :, 1] = torch.clip(offspring[:, :, 1], amt_bound[0], amt_bound[1])    # Clip amount

    # Replace old population with offspring for next generation
    population = offspring

    # (Optional) Adjust sigma dynamically
    successes = (fitness < 0).sum()  # Count successful offspring
    if successes / lambda_ > threshold:
        sigma_freq *= c_inc
        sigma_amt *= c_inc
    else:
        sigma_freq *= c_dec
        sigma_amt *= c_dec
